# Importation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import healpy as hp
import yaml

from qubic.lib.MapMaking.Qatmosphere_2d import AtmosphereMaps
from qubic.lib.Instrument.Qacquisition import QubicAcquisition
from qubic.lib.Instrument.Qinstrument import QubicInstrument
from qubic.lib.Qscene import QubicScene
from qubic.lib.Qsamplings import get_pointing, equ2gal, QubicSampling

from qubic.lib.MapMaking.Qcg import PCGAlgorithm
from pyoperators.iterative.core import AbnormalStopIteration

from pyoperators import MPI, BlockDiagonalOperator, DiagonalOperator, ReshapeOperator, Rotation3dOperator, Spherical2CartesianOperator, Cartesian2SphericalOperator

from pysimulators.interfaces.healpy import Spherical2HealpixOperator, HealpixConvolutionGaussianOperator

comm = MPI.COMM_WORLD
rank = comm.Get_rank() 

%matplotlib inline

Comparer (H.T H)⁻1 en local ou en galactic projeté sur une carte healpy 

# Atm class

In [ ]:
# Import simulation parameters
with open('params.yml', 'r') as file:
    params = yaml.safe_load(file) 

In [ ]:
# Call the class which build the atmosphere maps
atm = AtmosphereMaps(params)
qubic_dict = atm.qubic_dict

In [ ]:
wv_fluctuations = atm.rho_map
print(wv_fluctuations.shape[0])
plt.imshow(wv_fluctuations, cmap='jet', extent=[-params['size_atm'], params['size_atm'], -params['size_atm'], params['size_atm']])
plt.title('Water vapor density fluctuations')
plt.xlabel('m')
plt.ylabel('m')
plt.colorbar(label=r'$g/m^{3}$')

In [ ]:
print("Frequencies are : ", atm.frequencies, "GHz.")

In [ ]:
temp_fluctuations = np.zeros((wv_fluctuations.shape[0], wv_fluctuations.shape[1], 3))
temp_fluctuations[..., 0] = atm.get_temp_maps(wv_fluctuations)[0]
temp_fluctuations[..., 0] -= np.mean(temp_fluctuations[..., 0])

plt.imshow(temp_fluctuations[..., 0], cmap='jet', extent=[-params['size_atm'], params['size_atm'], -params['size_atm'], params['size_atm']])
plt.title('Temperature fluctuations')
plt.xlabel('m')
plt.ylabel('m')
plt.colorbar(label=r'$\mu K_{CMB}$')

In [ ]:
healpy_temp_fluctuations = np.zeros((hp.nside2npix(params['nside']), 3))
#healpy_temp_fluctuations[..., 0] = atm.get_healpy_atm_maps_2d(atm.get_temp_maps(wv_fluctuations))[0]

az, el = 211.92662822765195, 43.93466164903442
qubic_patch = np.array([[az, el]])
azel_coordinates = atm.get_azel_coordinates()
proj_hp = Cartesian2SphericalOperator('azimuth,elevation')(Rotation3dOperator("ZY'", az, 90 - el, degrees=True)(Spherical2CartesianOperator('azimuth,elevation')))
azel2hp = Spherical2HealpixOperator(params['nside'], 'azimuth,elevation')
hp_maps_index = azel2hp(proj_hp(azel_coordinates)).astype(int)
healpy_temp_fluctuations[hp_maps_index, 0] = atm.get_temp_maps(wv_fluctuations)[0].flatten()

index = np.where(healpy_temp_fluctuations!=0)[0]
healpy_temp_fluctuations[index, 0] -= np.mean(healpy_temp_fluctuations[index, 0])
min = np.min(healpy_temp_fluctuations[index, 0])
max = np.max(healpy_temp_fluctuations[index, 0])

hp.mollview(healpy_temp_fluctuations[..., 0], min=min, max=max, cmap='jet', title='Temperature fluctuations', unit=r'$µK_{CMB}$')
hp.gnomview(healpy_temp_fluctuations[..., 0], min=min, max=max, rot=qubic_patch, reso=15, cmap='jet', title='Temperature fluctuations', unit=r'$µK_{CMB}$')

In [ ]:
print('Angular speed', qubic_dict['angspeed'])
print('Delta azimtuh', qubic_dict['delta_az'])
print('Sweeps per elevation', qubic_dict['nsweeps_per_elevation'])
print('Angular speed psi', qubic_dict['angspeed_psi'])
print('Maximum psi', qubic_dict['maxppsi'])
print('latitude', qubic_dict['latitude'])
print('longitude', qubic_dict['longitude'])
print('Fix_azimuth', qubic_dict['fix_azimuth'])
print('period', qubic_dict['period'])
print('duration', qubic_dict['duration'])
print('Observation date', qubic_dict['date_obs'], type(qubic_dict['date_obs']))

# Scanning strategy

Next step is to build a scanning strategy following the qubic patch, but to perform the map-making in azimuth and elevation coordinates.

In [ ]:
print(qubic_dict['fix_azimuth'])

In [ ]:
### Random pointing
qubic_dict['random_pointing'] = True

### Sweepingpointing
qubic_dict['sweeping_pointing'] = False
qubic_dict['fix_azimuth']['apply'] = False 

qubic_dict['angspeed'] = 0.4
qubic_dict['delta_az'] = 20
qubic_dict['nsweeps_per_elevation'] = 1
qubic_dict['period'] = 1
qubic_dict['duration'] = 1
# npointings = 3600 * t_obs / period

### Repeat pointing
qubic_dict['repeat_pointing'] = False

qubic_dict['fix_azimuth']['apply'] = False

q_sampling_sweeping = get_pointing(qubic_dict)
center = np.array([0, -57])
print(q_sampling_sweeping)

In [ ]:
az, el = q_sampling_sweeping.azimuth, q_sampling_sweeping.elevation

fig, axs = plt.subplots(1, 5, figsize=(25, 5))

# Azimuth plot
axs[0].plot(az)
axs[0].set_title("Azimuth")
axs[0].set_xlabel("Time samples")
axs[0].set_ylabel("Angles (degrees)")

# Elevation plot
axs[1].plot(el)
axs[1].set_title("Elevation")
axs[1].set_xlabel("Time samples")
axs[1].set_ylabel("Angles (degrees)")

# Scanning strategy plot
axs[2].plot(az, el, '.')
axs[2].set_title("Scanning strategy")
axs[2].set_xlabel("Azimuth (degrees)")
axs[2].set_ylabel("Elevation (degrees)")

# Equatorial coordinates plot
axs[3].plot((q_sampling_sweeping.equatorial[:, 0] + 180) % 360 -180, q_sampling_sweeping.equatorial[:, 1], '.')
axs[3].set_title("Equatorial coordinates")
axs[3].set_xlabel("Right ascension (degrees)")
axs[3].set_ylabel("Declination (degrees)")

# Galactic coordinates plot
axs[4].plot(q_sampling_sweeping.galactic[:, 0], q_sampling_sweeping.galactic[:, 1], '.')
axs[4].set_title("Galactic coordinates")
axs[4].set_xlabel("Longitude (degrees)")
axs[4].set_ylabel("Latitude (degrees)")

fig.suptitle("Qubic Sampling")
plt.tight_layout()
plt.show()

In [ ]:
test_gal = np.zeros(hp.nside2npix(params['nside']))

index = np.array(Spherical2HealpixOperator(params['nside'], 'azimuth, elevation')(np.radians(q_sampling_sweeping.galactic)), dtype='int')
test_gal[index] = 1
hp.mollview(test_gal, title='Pointing in Galactic Coordinates', cmap='viridis')
hp.gnomview(test_gal, title='Pointing in Galactic Coordinates', cmap='viridis', reso=15, rot=equ2gal(center[0], center[1]))

# Scanning Strategy in local coordinates 

In [ ]:
q_sampling = QubicSampling(params['npointings'],
                           date_obs = qubic_dict['date_obs'],
                           period = qubic_dict['period'],
                           latitude = qubic_dict['latitude'],
                           longitude = qubic_dict['longitude'])

q_sampling.azimuth = q_sampling_sweeping.azimuth
q_sampling.elevation = q_sampling_sweeping.elevation
q_sampling.pitch = q_sampling_sweeping.pitch
q_sampling.angle_hwp = q_sampling_sweeping.angle_hwp

q_sampling.fix_az = True

In [ ]:
test_gal = np.zeros(hp.nside2npix(params['nside']))

index = np.array(Spherical2HealpixOperator(params['nside'], 'azimuth, elevation')(np.radians([q_sampling.azimuth, q_sampling.elevation]).T), dtype='int')
test_gal[index] = 1
hp.mollview(test_gal, title='Pointing in Local Coordinates', cmap='viridis')
hp.gnomview(test_gal, title='Pointing in Local Coordinates', cmap='viridis', reso=15, rot=(np.mean(q_sampling.azimuth), np.mean(q_sampling.elevation)))

print(np.mean(q_sampling.azimuth), np.mean(q_sampling.elevation))

# Build QUBIC instances

In [ ]:
q_instrument = QubicInstrument(qubic_dict)
q_scene  = QubicScene(qubic_dict)

q_acquisition = QubicAcquisition(q_instrument, q_sampling, q_scene, qubic_dict)

In [ ]:
coverage = q_acquisition.get_coverage()

covnorm = coverage / coverage.max()
seenpix = covnorm > params['coverage_cut']

qubic_patch = equ2gal(center[0], center[1])

In [ ]:
hp.mollview(coverage)

# Build QUBIC operators

In [ ]:
H_qubic = q_acquisition.get_operator()
R_qubic = ReshapeOperator(H_qubic.shapeout, H_qubic.shape[0])
invN_qubic = R_qubic(q_acquisition.get_invntt_operator(False, False)(R_qubic.T))
C_qubic = HealpixConvolutionGaussianOperator(fwhm=0.00628339517640407)
d_qubic = H_qubic(C_qubic(healpy_temp_fluctuations)).ravel()

In [ ]:
H = R_qubic(H_qubic)
invN = invN_qubic
d = d_qubic

# Map-Making

In [ ]:
A = H.T * invN * H
b = H.T * invN * d
true_maps = C_qubic(healpy_temp_fluctuations)
x0 = true_maps * 0

In [ ]:
# Preconditioner
stacked_dptdp_inv = np.zeros((1,12*params['nside']**2))

D = H.operands[2]
P = H.operands[-1]
sh = P.matrix.data.index.shape
no_det = 992
point_per_det = int(sh[0] / no_det)
mapPtP_perdet_seq = np.zeros((no_det, 12 * params['nside']**2))
sample_ranges = [(det * point_per_det, (det + 1) * point_per_det) for det in range(no_det)]
for det, (start, end) in enumerate(sample_ranges):
    indices = P.matrix.data.index[start:end, :]  
    weights = P.matrix.data.r11[start:end, :]
    flat_indices = indices.ravel()
    flat_weights = weights.ravel()

    mapPitPi = np.zeros(12 * params['nside']**2)
    np.add.at(mapPitPi, flat_indices, flat_weights**2)

    mapPtP_perdet_seq[det, :] = mapPitPi
D_elements = D.data
D_sq = D_elements**2
mapPtP_seq_scaled = D_sq[:, np.newaxis] * mapPtP_perdet_seq 
dptdp = mapPtP_seq_scaled.sum(axis = 0)
dptdp_inv = 1 / dptdp
dptdp_inv[np.isinf(dptdp_inv)] = 0.
stacked_dptdp_inv[0] = dptdp_inv
M = BlockDiagonalOperator( \
            [DiagonalOperator(ci, broadcast='rightward') for ci in stacked_dptdp_inv],
            new_axisin=0)

In [ ]:
# Run PCG
algo = PCGAlgorithm(
    A,
    b,
    comm,
    x0=x0,
    tol=1e-10,
    maxiter=200,
    disp=True,
    M=M,
    center=[0, -57],
    reso=15,
    seenpix=seenpix,
    input=true_maps,
)
try:
    output = algo.run()
    success = True
    message = 'Success'
except AbnormalStopIteration as e:
    output = algo.finalize()
    success = False
    message = str(e)

In [ ]:
plt.plot(output['convergence'])
plt.yscale('log')
plt.xlabel('Iteration')
plt.ylabel('Convergence')

In [ ]:
plt.figure(figsize=(12, 25))
k=1

istk = 0
stk = ['I', 'Q', 'U']

sigma = np.std(true_maps[seenpix, istk])
hp.mollview(true_maps[:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k), title=f'{stk[istk]} - Input', notext=True)
hp.mollview(output['x'][:, istk], min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k+1), title=f'{stk[istk]} - Output', notext=True)
hp.mollview(output['x'][:, istk] - true_maps[:, istk], cmap='jet', sub=(1, 3, k+2), title=f'{stk[istk]} - Residual', notext=True)
k+=3

In [ ]:
plt.figure(figsize=(12, 25))
k=1

istk = 0

center_azel = np.array([np.mean(q_sampling.azimuth), np.mean(q_sampling.elevation)])
print(center_azel)
reso = 25

sigma = np.std(true_maps[seenpix, istk])
hp.gnomview(true_maps[:, istk], reso=reso, rot=center_azel, min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k), title=f'{stk[istk]} - Input', notext=True)
hp.gnomview(output['x'][:, istk], reso=reso, rot=center_azel, min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k+1), title=f'{stk[istk]} - Output', notext=True)
hp.gnomview(output['x'][:, istk] - true_maps[:, istk], reso=reso, rot=center_azel, cmap='jet', sub=(1, 3, k+2), title=f'{stk[istk]} - Residual', notext=True)
k+=3

In [ ]:
plt.figure(figsize=(12, 25))
k=1

true_maps[~seenpix, :] = hp.UNSEEN
output['x'][~seenpix, :] = hp.UNSEEN

istk = 0

center_azel = np.array([np.mean(q_sampling.azimuth), np.mean(q_sampling.elevation)])
reso = 20

sigma = np.std(true_maps[seenpix, istk])
hp.gnomview(true_maps[:, istk], reso=reso, rot=center_azel, min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k), title=f'{stk[istk]} - Input', notext=True)
hp.gnomview(output['x'][:, istk], reso=reso, rot=center_azel, min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k+1), title=f'{stk[istk]} - Output', notext=True)
hp.gnomview(output['x'][:, istk] - true_maps[:, istk], reso=reso, rot=center_azel, cmap='jet', sub=(1, 3, k+2), title=f'{stk[istk]} - Residual', notext=True)
k+=3

In [ ]:
plt.figure(figsize=(12, 25))
k=1

true_maps[~seenpix, :] = hp.UNSEEN
output['x'][~seenpix, :] = hp.UNSEEN

istk = 1

center_azel = np.array([np.mean(q_sampling.azimuth), np.mean(q_sampling.elevation)])
reso = 20

sigma = np.std(true_maps[seenpix, istk])
hp.gnomview(true_maps[:, istk], reso=reso, rot=center_azel, min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k), title=f'{stk[istk]} - Input', notext=True)
hp.gnomview(output['x'][:, istk], reso=reso, rot=center_azel, min=np.min(true_maps[seenpix, istk]), max=np.max(true_maps[seenpix, istk]), cmap='jet', sub=(1, 3, k+1), title=f'{stk[istk]} - Output', notext=True)
hp.gnomview(output['x'][:, istk] - true_maps[:, istk], reso=reso, rot=center_azel, cmap='jet', sub=(1, 3, k+2), title=f'{stk[istk]} - Residual', notext=True)
k+=3